In [7]:
import os
import random
import shutil
import string

In [2]:
def transfer_files(fold_first, fold_second, class_num): # микс нового мини-датасета со старым
    # Проверяем корректность номера класса
    if class_num not in range(10):
        raise ValueError("Номер класса должен быть от 0 до 9")

    # Получаем список файлов в первой папке
    first_folder_files = os.listdir(fold_first)

    # Фильтруем только изображения (игнорируем текстовые файлы)
    first_folder_images = [f for f in first_folder_files
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Вычисляем 10% от текущего количества изображений
    ten_percent = max(1, len(first_folder_images) // 10)  # минимум 1 файл

    # Пути к папкам во второй папке
    train_images_path = os.path.join(fold_second, 'train', 'images')
    train_labels_path = os.path.join(fold_second, 'train', 'labels')

    # Получаем список всех изображений нужного класса во второй папке
    class_images = []
    for image_file in os.listdir(train_images_path):
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(train_labels_path, label_file)

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                content = f.read().strip().split()
                if content and content[0] == str(class_num):
                    class_images.append((image_file, label_file))

    # Выбираем случайные 10% изображений нужного класса
    selected_class_files = random.sample(class_images, min(ten_percent, len(class_images)))

    # Переносим выбранные файлы в первую папку
    for image_file, label_file in selected_class_files:
        src_image = os.path.join(train_images_path, image_file)
        dst_image = os.path.join(fold_first, image_file)
        shutil.copy2(src_image, dst_image)

        src_label = os.path.join(train_labels_path, label_file)
        dst_label = os.path.join(fold_first, label_file)
        shutil.copy2(src_label, dst_label)

    # Обновляем количество изображений в первой папке после добавления 10%
    new_count = len(first_folder_images) + len(selected_class_files)

    # Теперь переносим файлы других классов (9 * исходное количество)
    other_class_count = len(first_folder_images)  # исходное количество

    # Собираем файлы других классов
    other_class_files = []
    for image_file in os.listdir(train_images_path):
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(train_labels_path, label_file)

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                content = f.read().strip().split()
                if content and content[0] != str(class_num):
                    other_class_files.append((image_file, label_file))

    # Выбираем случайные файлы других классов
    selected_other_files = random.sample(other_class_files,
                                         min(other_class_count * 9, len(other_class_files)))

    # Переносим выбранные файлы других классов
    for image_file, label_file in selected_other_files:
        src_image = os.path.join(train_images_path, image_file)
        dst_image = os.path.join(fold_first, image_file)
        shutil.copy2(src_image, dst_image)

        src_label = os.path.join(train_labels_path, label_file)
        dst_label = os.path.join(fold_first, label_file)
        shutil.copy2(src_label, dst_label)

    print(f"Перенесено {len(selected_class_files)} файлов класса {class_num}")
    print(f"Перенесено {len(selected_other_files)} файлов других классов")

In [ ]:
fold_first = "test_images_new"
fold_second = "test_images_old"
transfer_files(fold_first, fold_second, 8) # папка с мини-датасетом, папка со старым датасетом, номер класса в мини-датасете

In [4]:
def generate_random_name(length=8): # функция случайного переименования файлов для равномерности распределения при перемещении в папки train и test 
    """Генерирует случайное имя заданной длины."""

    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(length))


def rename_matching_files(directory):
    """Находит и переименовывает пары файлов с одинаковыми именами, но разными расширениями."""
    # Собираем все файлы в директории
    files = os.listdir(directory)

    # Создаем словарь: имя -> список расширений
    name_dict = {}
    for file in files:
        name, ext = os.path.splitext(file)
        if name not in name_dict:
            name_dict[name] = []
        name_dict[name].append(ext.lower())

    # Переименовываем подходящие пары
    for name, exts in name_dict.items():
        # Проверяем, есть ли оба расширения (.txt и .jpg/.jpeg)
        has_txt = '.txt' in exts
        has_jpg = '.jpg' in exts or '.jpeg' in exts

        if has_txt and has_jpg:
            # Генерируем новое имя
            new_name = generate_random_name()

            # Переименовываем .txt файл
            txt_ext = '.txt'
            old_txt_path = os.path.join(directory, name + txt_ext)
            new_txt_path = os.path.join(directory, new_name + txt_ext)
            os.rename(old_txt_path, new_txt_path)

            # Переименовываем .jpg/.jpeg файл
            jpg_ext = '.jpg' if '.jpg' in exts else '.jpeg'
            old_jpg_path = os.path.join(directory, name + jpg_ext)
            new_jpg_path = os.path.join(directory, new_name + jpg_ext)
            os.rename(old_jpg_path, new_jpg_path)

            print(f'Rename: {name}{txt_ext} и {name}{jpg_ext} -> {new_name}{txt_ext} and {new_name}{jpg_ext}')

In [ ]:
fold = "test_images_old"
if os.path.isdir(fold):
    rename_matching_files(fold)
    print("Готово!")
else:
    print("Указанная директория не существует.")

In [ ]:
def split_data(source_dir, test_ratio=0.2): # распределение фото и лейблов на train и test, затем на images и labels
    # Создаем папки test и train
    test_dir = os.path.join(source_dir, 'test')
    train_dir = os.path.join(source_dir, 'train')

    # Создаем подпапки images и labels в test и train
    test_images_dir = os.path.join(test_dir, 'images')
    test_labels_dir = os.path.join(test_dir, 'labels')
    train_images_dir = os.path.join(train_dir, 'images')
    train_labels_dir = os.path.join(train_dir, 'labels')

    for dir_path in [test_images_dir, test_labels_dir, train_images_dir, train_labels_dir]:
        os.makedirs(dir_path, exist_ok=True)

    # Получаем список всех изображений
    image_extensions = ('.jpg', '.jpeg', '.png')
    images = [f for f in os.listdir(source_dir) if f.lower().endswith(image_extensions)]

    # Перемешиваем для случайного выбора
    random.shuffle(images)

    # Вычисляем количество файлов для теста
    test_count = int(len(images) * test_ratio)

    # Разделяем на тестовые и тренировочные
    test_images = images[:test_count]
    train_images = images[test_count:]

    # Функция для перемещения файлов

    def move_files(file_list, images_dest, labels_dest):
        for image_file in file_list:
            # Получаем базовое имя без расширения
            base_name = os.path.splitext(image_file)[0]

            # Находим соответствующий txt файл
            txt_file = f"{base_name}.txt"

            # Проверяем существование txt файла
            if not os.path.exists(os.path.join(source_dir, txt_file)):
                print(f"Warning: No corresponding txt file for {image_file}")
                continue

            # Перемещаем изображение
            shutil.move(
                os.path.join(source_dir, image_file),
                os.path.join(images_dest, image_file)
            )

            # Перемещаем текстовый файл
            shutil.move(
                os.path.join(source_dir, txt_file),
                os.path.join(labels_dest, txt_file)
            )

    # Перемещаем тестовые файлы
    move_files(test_images, test_images_dir, test_labels_dir)

    # Перемещаем тренировочные файлы
    move_files(train_images, train_images_dir, train_labels_dir)

    print(f"Data split completed. {len(test_images)} files moved to test, {len(train_images)} files moved to train.")

In [ ]:
fold = "test_images_new"
if not os.path.isdir(fold):
    print(f"Error: Directory '{fold}' does not exist.")
split_data(fold)